# Python `tspart` Module Usage Example

In [ ]:
import os
import tspart
from IPython.display import display

In [ ]:
email = input("Please enter your email address (for sending requests to the Neos online solver):").strip()

Import the image, get it's size, and split into CMYK channels.

In [ ]:
img = tspart.load_image_as_array(os.path.join("tests", "baboon.png"))
size = tspart.image_array_size(img)
cmyk = tspart.split_cmyk(img)

Stipple the images.

In [ ]:
cmyk_points = tspart.voronoi.stipple_image_multi(cmyk, points=500)

Save the stippled points to disk (in order to resume after a failure).

In [ ]:
tspart.save_routes(os.path.join("tests", "tspart_raw.json"), cmyk_points, size)

Solve the TSP problems with a free online solver.

This will take a long time.

In [ ]:
neos = tspart.neos.get_client()
jobs = tspart.neos.submit_solves(neos, email, cmyk_points)
cmyk_routes = tspart.neos.get_solves_blocking(neos, jobs, cmyk_points)

Save the routes locally (in order to resume after a failure).

Additionally, delete the pre-TSP data.

In [ ]:
tspart.save_routes(os.path.join("tests", "tspart.json"), cmyk_routes, size)
os.remove(os.path.join("tests", "tspart_raw.json"))

Draw a bitmap image.

In [ ]:
drawn_image = tspart.draw_cmyk_routes(cmyk_routes, line_width=7.5, images=cmyk, scale=4)

width = 600

ratio = size[0] / size[1]
new_size = (
    width,
    int(round(width / ratio))
)

display(drawn_image.resize(new_size))

Save the image.

In [ ]:
drawn_image.save(os.path.join("tests", "tspart.png"))

# Tools:

Load raw (pre-TSP) points from disk.

In [ ]:
cmyk_points, size = tspart.load_routes(os.path.join("tests", "tspart_raw.json"))

Load precomputed route data from disk.

In [ ]:
cmyk_routes, size = tspart.load_routes(os.path.join("tests", "tspart.json"))

Run a heuristic (non-exact) solve locally with a 1-hour timeout for each channel.

In [ ]:
cmyk_routes = tspart.tsp.heuristic_solves(cmyk_points, time_limit_minutes=60, symmetric=True)

Cancel Neos solves.

In [ ]:
tspart.neos.cancel_solves(neos, jobs)

Manually recover Neos solves with jobs numbers and passwords.

In [ ]:
jobs = [
    (13820283, "ckmupfHR"),
    (13820284, "GeQdiMoA"),
    (13820285, "RacOsQuM"),
    (13820286, "OyWRtNBp")
]
neos = tspart.neos.get_client()
cmyk_routes = tspart.neos.get_solves_blocking(neos, jobs, cmyk_points)

Print the job numbers and passwords for the current jobs.

In [ ]:
print(jobs)

# Setup Utilities

Install the `tspart` module.

In [ ]:
!pip install --upgrade --no-cache-dir tspart

Download the test image.

In [ ]:
!curl https://raw.githubusercontent.com/nimaid/python-tspart/main/tests/baboon.png -o baboon.png
import shutil
shutil.move("baboon.png", os.path.join("tests", "baboon.png"))